In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
def rnd(x):
    ## round function
    dcm = x%1
    itg = x-dcm
    dx = 0 if dcm<.5 else 1
    return itg+dx

In [3]:
def cost(lv,a,b):
    ## cost calculator
    base = 1 if lv==0 else lv
    return rnd((base+a+b)*.5)

In [4]:
def pinpoint_num(cell):
    ## extract numbers, then save it as an array
    chars = re.findall(r"[-+]?\d*\.\d+|\d+", cell)
    if len(chars)>0:
        temp = []
        for c in chars:
            temp.append(float(c))
    else:
        temp = ['?'] ## There is no number
    return np.array(temp)

In [5]:
def pinpoint_unit(cell):
    ## extract units, then save it as an array
    chars = re.findall(r"\%| sec|m", cell)
    if len(chars)==0:
        chars=[''] ## There is no unit
    return np.array(chars)

In [6]:
def calc_value(name,data):
    ## make an array which consists with ratios
    vv = []
    for rk in range(9):
        cost = data.ideal_cost[rk]
        cell = data.loc[rk,name]
        effects = pinpoint_num(cell) if name!='Resilience' else 100-pinpoint_num(cell)
        units = pinpoint_unit(cell)
        
        temp = ''
        for i,e in enumerate(effects):
            if e!='?':
                if i==0:
                    temp += "%g%s"%(e/cost,units[i])
                else:
                    temp += "/%g%s"%(e/cost,units[i])
            else:
                temp = '?'
        vv.append(temp)
    return vv

In [7]:
def show_value(data):
    ## show the processed dataframe
    dv = pd.DataFrame(index=range(9))

    df=data
    for name in df.columns[1:]:
        tt = calc_value(name,df)
        dv[name] = tt
    return dv

In [8]:
IC = {'ideal_cost':[4,5,6,7,9,11,14,17,21]}

### Attack Chips

In [9]:
da = pd.DataFrame(IC)
da['Charge_Attack'] = ['120%','140%','160%','180%','200%','250%','300%','350%','400%']
da['Counter'] = ['0%','10%','30%','60%','80%','100%','150%','200%','250%']
da['Critical_Up'] = ['1%','2%','3%','4%','6%','8%','10%','15%','30%']
da['Down-Attack_Up'] = ['2%','4%','8%','10%','15%','20%','50%','80%','100%']
da['Last_Stand'] = ['5%','10%','15%','20%','30%','50%','60%','80%','100%']
da['Ranged_Attack_Up'] = ['2%','4%','8%','10%','15%','20%','50%','80%','100%']
da['Shockwave'] = ['Weak','S. Weak','Normal','S. Strong','F. Strong','Strong','Powerful','P. Powerful','V. Powerful']
da['Weapon_Attack_Up'] = ['2%','4%','8%','10%','15%','20%','50%','80%','100%']
da

,ideal_cost,Charge_Attack,Counter,Critical_Up,Down-Attack_Up,Last_Stand,Ranged_Attack_Up,Shockwave,Weapon_Attack_Up
0,4,120%,0%,1%,2%,5%,2%,Weak,2%
1,5,140%,10%,2%,4%,10%,4%,S. Weak,4%
2,6,160%,30%,3%,8%,15%,8%,Normal,8%
3,7,180%,60%,4%,10%,20%,10%,S. Strong,10%
4,9,200%,80%,6%,15%,30%,15%,F. Strong,15%
5,11,250%,100%,8%,20%,50%,20%,Strong,20%
6,14,300%,150%,10%,50%,60%,50%,Powerful,50%
7,17,350%,200%,15%,80%,80%,80%,P. Powerful,80%
8,21,400%,250%,30%,100%,100%,100%,V. Powerful,100%


In [10]:
show_value(da)

,Charge_Attack,Counter,Critical_Up,Down-Attack_Up,Last_Stand,Ranged_Attack_Up,Shockwave,Weapon_Attack_Up
0,30%,0%,0.25%,0.5%,1.25%,0.5%,?,0.5%
1,28%,2%,0.4%,0.8%,2%,0.8%,?,0.8%
2,26.6667%,5%,0.5%,1.33333%,2.5%,1.33333%,?,1.33333%
3,25.7143%,8.57143%,0.571429%,1.42857%,2.85714%,1.42857%,?,1.42857%
4,22.2222%,8.88889%,0.666667%,1.66667%,3.33333%,1.66667%,?,1.66667%
5,22.7273%,9.09091%,0.727273%,1.81818%,4.54545%,1.81818%,?,1.81818%
6,21.4286%,10.7143%,0.714286%,3.57143%,4.28571%,3.57143%,?,3.57143%
7,20.5882%,11.7647%,0.882353%,4.70588%,4.70588%,4.70588%,?,4.70588%
8,19.0476%,11.9048%,1.42857%,4.7619%,4.7619%,4.7619%,?,4.7619%


### Defense Chips

In [11]:
db = pd.DataFrame(IC)
db['Anti_Chain_Damage'] = ['0.5 sec','1 sec','1.5 sec','2 sec','2.5 sec','3 sec','4 sec','5 sec','6 sec']
db['Auto-Heal'] = ['0.60%','1.20%','2.40%','3.60%','4.80%','7.20%','9.60%','12%','18%']
db['Damage_Absorb'] = ['5%/10%','10%/20%','15%/30%','20%/40%','25%/50%','30%/80%','35%/100%','40%/150%','50%/200%']
db['Deadly_Heal'] = ['5%','10%','20%','30%','40%','50%','60%','80%','100%']
db['Max_HP_Up'] = ['5%','10%','15%','20%','25%','30%','50%','80%','100%']
db['Melee_Defense'] = ['2%','4%','8%','10%','15%','20%','30%','60%','80%']
db['Offensive_Heal'] = ['2%','5%','10%','15%','20%','30%','50%','80%','100%']
db['Ranged_Defense'] = ['2%','4%','8%','10%','15%','20%','30%','60%','80%']
db['Reset'] = ['5%/10%','5%/20%','10%/20%','15%/30%','20%/40%','25%/50%','30%/60%','40%/70%','50%/80%']
db['Resilience'] = ['80%','70%','60%','50%','40%','30%','20%','10%','5%']
db

,ideal_cost,Anti_Chain_Damage,Auto-Heal,Damage_Absorb,Deadly_Heal,Max_HP_Up,Melee_Defense,Offensive_Heal,Ranged_Defense,Reset,Resilience
0,4,0.5 sec,0.60%,5%/10%,5%,5%,2%,2%,2%,5%/10%,80%
1,5,1 sec,1.20%,10%/20%,10%,10%,4%,5%,4%,5%/20%,70%
2,6,1.5 sec,2.40%,15%/30%,20%,15%,8%,10%,8%,10%/20%,60%
3,7,2 sec,3.60%,20%/40%,30%,20%,10%,15%,10%,15%/30%,50%
4,9,2.5 sec,4.80%,25%/50%,40%,25%,15%,20%,15%,20%/40%,40%
5,11,3 sec,7.20%,30%/80%,50%,30%,20%,30%,20%,25%/50%,30%
6,14,4 sec,9.60%,35%/100%,60%,50%,30%,50%,30%,30%/60%,20%
7,17,5 sec,12%,40%/150%,80%,80%,60%,80%,60%,40%/70%,10%
8,21,6 sec,18%,50%/200%,100%,100%,80%,100%,80%,50%/80%,5%


In [12]:
show_value(db)

,Anti_Chain_Damage,Auto-Heal,Damage_Absorb,Deadly_Heal,Max_HP_Up,Melee_Defense,Offensive_Heal,Ranged_Defense,Reset,Resilience
0,0.125 sec,0.15%,1.25%/2.5%,1.25%,1.25%,0.5%,0.5%,0.5%,1.25%/2.5%,5%
1,0.2 sec,0.24%,2%/4%,2%,2%,0.8%,1%,0.8%,1%/4%,6%
2,0.25 sec,0.4%,2.5%/5%,3.33333%,2.5%,1.33333%,1.66667%,1.33333%,1.66667%/3.33333%,6.66667%
3,0.285714 sec,0.514286%,2.85714%/5.71429%,4.28571%,2.85714%,1.42857%,2.14286%,1.42857%,2.14286%/4.28571%,7.14286%
4,0.277778 sec,0.533333%,2.77778%/5.55556%,4.44444%,2.77778%,1.66667%,2.22222%,1.66667%,2.22222%/4.44444%,6.66667%
5,0.272727 sec,0.654545%,2.72727%/7.27273%,4.54545%,2.72727%,1.81818%,2.72727%,1.81818%,2.27273%/4.54545%,6.36364%
6,0.285714 sec,0.685714%,2.5%/7.14286%,4.28571%,3.57143%,2.14286%,3.57143%,2.14286%,2.14286%/4.28571%,5.71429%
7,0.294118 sec,0.705882%,2.35294%/8.82353%,4.70588%,4.70588%,3.52941%,4.70588%,3.52941%,2.35294%/4.11765%,5.29412%
8,0.285714 sec,0.857143%,2.38095%/9.52381%,4.7619%,4.7619%,3.80952%,4.7619%,3.80952%,2.38095%/3.80952%,4.52381%


### Support Chips

In [13]:
dc = pd.DataFrame(IC)
dc['Auto-Use_Item'] = ['10%','15%','20%','30%','40%','50%','60%','80%','100%']
dc['Drop_Rate_Up'] = ['10%','20%','30%','40%','50%','60%','70%','80%','90%']
dc['EXP_Gain_Up'] = ['2%','4%','8%','10%','20%','30%','50%','80%','100%']
dc['Evade_Range_Up'] = ['10%','20%','40%','60%','80%','100%','120%','160%','200%']
dc['Fast_Cooldown'] = ['2%','4%','8%','10%','15%','20%','25%','35%','50%']
dc['Moving_Speed_Up'] = ['2%','4%','8%','10%','12%','14%','16%','18%','20%']
dc['Overclock'] = ['0.8 sec','1.5 sec','2 sec','2.5 sec','3 sec','3.5 sec','4 sec','4.5 sec','5.5 sec']
dc['Taunt_Up'] = ['180%','200%','240%','280%','300%','350%','400%','450%','500%']
dc['Vengeance'] = ['5%/10%','10%/20%','15%/30%','20%/40%','25%/50%','30%/80%','35%/100%','40%/150%','50%/200%']
dc

,ideal_cost,Auto-Use_Item,Drop_Rate_Up,EXP_Gain_Up,Evade_Range_Up,Fast_Cooldown,Moving_Speed_Up,Overclock,Taunt_Up,Vengeance
0,4,10%,10%,2%,10%,2%,2%,0.8 sec,180%,5%/10%
1,5,15%,20%,4%,20%,4%,4%,1.5 sec,200%,10%/20%
2,6,20%,30%,8%,40%,8%,8%,2 sec,240%,15%/30%
3,7,30%,40%,10%,60%,10%,10%,2.5 sec,280%,20%/40%
4,9,40%,50%,20%,80%,15%,12%,3 sec,300%,25%/50%
5,11,50%,60%,30%,100%,20%,14%,3.5 sec,350%,30%/80%
6,14,60%,70%,50%,120%,25%,16%,4 sec,400%,35%/100%
7,17,80%,80%,80%,160%,35%,18%,4.5 sec,450%,40%/150%
8,21,100%,90%,100%,200%,50%,20%,5.5 sec,500%,50%/200%


In [14]:
show_value(dc)

,Auto-Use_Item,Drop_Rate_Up,EXP_Gain_Up,Evade_Range_Up,Fast_Cooldown,Moving_Speed_Up,Overclock,Taunt_Up,Vengeance
0,2.5%,2.5%,0.5%,2.5%,0.5%,0.5%,0.2 sec,45%,1.25%/2.5%
1,3%,4%,0.8%,4%,0.8%,0.8%,0.3 sec,40%,2%/4%
2,3.33333%,5%,1.33333%,6.66667%,1.33333%,1.33333%,0.333333 sec,40%,2.5%/5%
3,4.28571%,5.71429%,1.42857%,8.57143%,1.42857%,1.42857%,0.357143 sec,40%,2.85714%/5.71429%
4,4.44444%,5.55556%,2.22222%,8.88889%,1.66667%,1.33333%,0.333333 sec,33.3333%,2.77778%/5.55556%
5,4.54545%,5.45455%,2.72727%,9.09091%,1.81818%,1.27273%,0.318182 sec,31.8182%,2.72727%/7.27273%
6,4.28571%,5%,3.57143%,8.57143%,1.78571%,1.14286%,0.285714 sec,28.5714%,2.5%/7.14286%
7,4.70588%,4.70588%,4.70588%,9.41176%,2.05882%,1.05882%,0.264706 sec,26.4706%,2.35294%/8.82353%
8,4.7619%,4.28571%,4.7619%,9.52381%,2.38095%,0.952381%,0.261905 sec,23.8095%,2.38095%/9.52381%


### Hacking Chips

In [15]:
dd = pd.DataFrame(IC)
dd['Combust'] = ['','3.5 sec','4 sec','4.5 sec','5 sec','5.5 sec','6 sec','6.5 sec','7 sec']
dd['Heal_Drops_Up'] = ['','','30%','40%','50%','60%','70%','90%','100%']
dd['Hijack_Boost'] = ['1','2','3','4','5','6','7','8','9']
dd['Stun'] = ['5m/3 sec','10m/3 sec','10m/3.5 sec','15m/3.5 sec','15m/4 sec','20m/4 sec','20m/4.5 sec','25m/4.5 sec','25m/5 sec']
dd

,ideal_cost,Combust,Heal_Drops_Up,Hijack_Boost,Stun
0,4,,,1,5m/3 sec
1,5,3.5 sec,,2,10m/3 sec
2,6,4 sec,30%,3,10m/3.5 sec
3,7,4.5 sec,40%,4,15m/3.5 sec
4,9,5 sec,50%,5,15m/4 sec
5,11,5.5 sec,60%,6,20m/4 sec
6,14,6 sec,70%,7,20m/4.5 sec
7,17,6.5 sec,90%,8,25m/4.5 sec
8,21,7 sec,100%,9,25m/5 sec


In [16]:
show_value(dd)

,Combust,Heal_Drops_Up,Hijack_Boost,Stun
0,?,?,0.25,1.25m/0.75 sec
1,0.7 sec,?,0.4,2m/0.6 sec
2,0.666667 sec,5%,0.5,1.66667m/0.583333 sec
3,0.642857 sec,5.71429%,0.571429,2.14286m/0.5 sec
4,0.555556 sec,5.55556%,0.555556,1.66667m/0.444444 sec
5,0.5 sec,5.45455%,0.545455,1.81818m/0.363636 sec
6,0.428571 sec,5%,0.5,1.42857m/0.321429 sec
7,0.382353 sec,5.29412%,0.470588,1.47059m/0.264706 sec
8,0.333333 sec,4.7619%,0.428571,1.19048m/0.238095 sec


### In short, the chips for which +8 is not optimal are:
Anti Chain Damage: Optimal at +3 +3 +3 \
Resilience: Optimal at +3 +3 \
Moving Speed Up: Optimal at +3 +3 \
Drop Rate Up: Optimal at +4 +3 \
Overclock: Optimal at +4 +3 \
Taunt Up: Optimal at +3 +2 +0 \
Hijack Boost: Optimal at +4 +3 \
Stun: Optimal at +3 +1 \
Combust: Optimal at +1 +1 \
Heal Drops Up: Optimal at +4 +4

### Chip Drop List

In [17]:
idx = ['Small Stubby', 'Multi-tier Type', 'Small Biped', 'Medium Quadruped', 'Multi-leg Medium Model'
      , 'Goliath Biped', 'Reverse-jointed Goliath', 'Small Flyer', 'Medium Flyer', 'Small Exploder'
      , 'Medium Exploder', 'Small Sphere', 'Linked Sphere']
dcdl = pd.DataFrame(index=idx)
CR = ['Hijack Boost', 'Auto-use Item', 'Critical Up/Hijack Boost', 'Weapon Attack Up', 'Weapon Attack Up'
     , 'Down-Attack Up', '', 'Ranged Attack Up', 'Ranged Attack Up', 'Moving Speed Up'
      , 'Charge Attack / Moving Speed Up', 'Auto-Heal', 'Auto-Heal']
De = ['Combust / Counter', 'Auto-use Item', 'Shock Wave / Moving Speed Up', 'N/A', '', '', 'N/A'
     , 'Ranged Attack Up / Taunt Up', '', '', '', 'N/A', 'N/A']
AP = ['EXP Up', 'N/A', 'Vengeance', 'N/A', 'N/A', 'N/A', 'N/A', 'Drop Rate Up / Heal Drops Up', ''
     , 'N/A', 'N/A', 'N/A', 'N/A']
FK = ['Anti Chain Damage / Deadly Heal', 'N/A', 'Deadly Heal / Max HP Up'
      , 'Melee Defense / Offensive Heal / Deadly Heal', 'N/A', 'Resilience', 'N/A', 'Ranged Attack Up'
     , '', 'N/A', 'N/A', 'N/A', 'Auto-Heal']
Fa = ['Fast Cooldown', 'N/A', 'Last Stand', 'N/A', 'Weapon Attack Up', '', 'N/A', 'Ranged Attack Up', '', ''
     , 'N/A', 'Auto-Heal', 'N/A']
dcdl['City Ruins'] = CR
dcdl['Desert'] = De
dcdl['Amusement Park'] = AP
dcdl['Forest Kingdom'] = FK
dcdl['Factory'] = Fa
dcdl

,City Ruins,Desert,Amusement Park,Forest Kingdom,Factory
Small Stubby,Hijack Boost,Combust / Counter,EXP Up,Anti Chain Damage / Deadly Heal,Fast Cooldown
Multi-tier Type,Auto-use Item,Auto-use Item,N/A,N/A,N/A
Small Biped,Critical Up/Hijack Boost,Shock Wave / Moving Speed Up,Vengeance,Deadly Heal / Max HP Up,Last Stand
Medium Quadruped,Weapon Attack Up,N/A,N/A,Melee Defense / Offensive Heal / Deadly Heal,N/A
Multi-leg Medium Model,Weapon Attack Up,,N/A,N/A,Weapon Attack Up
Goliath Biped,Down-Attack Up,,N/A,Resilience,
Reverse-jointed Goliath,,N/A,N/A,N/A,N/A
Small Flyer,Ranged Attack Up,Ranged Attack Up / Taunt Up,Drop Rate Up / Heal Drops Up,Ranged Attack Up,Ranged Attack Up
Medium Flyer,Ranged Attack Up,,,,
Small Exploder,Moving Speed Up,,N/A,N/A,
